In [15]:
import json
import pandas as pd
from scipy.stats import zscore
from glob import glob
import plotly.express as px

def _load_study(key) -> dict:
    with open(key, 'r') as f:
        data = json.load(f)
    return data['FullStudy']['Study']

# now we have a batch of studies in memory, which we could write to parquet? or just keep on hand? Idk we'll figure it out

def protocol_query(limit=1000) -> pd.DataFrame:
    DATA_BASE = "/Users/parker/Development/clinical-trials-dev/data"
    keys = glob(f"{DATA_BASE}/AllAPIJSON/*/*.json") # download and unpack from ClinicalTrials.gov
    study = pd.DataFrame([_load_study(key) for key in keys[0:limit]])
    protocol = pd.json_normalize(study['ProtocolSection']) # Der
    protocol.columns = [col.split(".")[-1] for col in protocol.columns] # yass
    return protocol

def protocol_feature_query(limit=1000) -> pd.DataFrame:
    protocol = protocol_query(limit=limit)
    feature = pd.DataFrame()
    feature_map = {
        "_id":"NCTId",
        '_org_name':'OrgFullName',
        # '_org_class':'OrgClass',
        '_sponsor_name':'LeadSponsorName',
        # '_sponsor_class':'LeadSponsorClass',
        # '_collaborator':'Collaborator',
        # '_type':'StudyType', # They were all intervernational after the dropna below
        # '_condition':'Condition',
        '_location':'Location',
        # '_eligibility':'EligibilityCriteria',
        '_enrollment': "EnrollmentCount",
        '_status':'OverallStatus',
        '_phase':'Phase',
        '_arm': 'ArmGroup',
        '_random': 'DesignAllocation'
    }

    for k, v in feature_map.items():
        feature[k] = protocol[v]

    feature = feature.dropna(subset=['_enrollment','_phase','_random']) # this cleans up a lot
    
    feature['_location_count'] = protocol['Location'].dropna().apply(lambda x: len(x))
    feature['_arm_count'] = protocol['ArmGroup'].dropna().apply(lambda x: len(x))
    feature['_start_yr'] = protocol['StartDate'].apply(lambda x: int(x.split(" ")[-1]) if pd.isna(x) == False else None)
    feature['_end_yr'] = protocol['PrimaryCompletionDate'].apply(lambda x: int(x.split(" ")[-1]) if pd.isna(x) == False else None)
    feature['_last_yr'] = protocol['LastUpdateSubmitDate'].apply(lambda x: int(x.split(" ")[-1]) if pd.isna(x) == False else None)
    
    feature["_phase"] = feature["_phase"].dropna().apply(lambda x: x[0])
    feature['_enrollment_z'] = zscore(feature['_enrollment'].astype(int)) 
    feature['_location_count_z'] = zscore(feature['_location_count'].fillna(0)) 
    feature['_arm_count_z'] = zscore(feature['_arm_count'].fillna(0))
    
    return feature.reset_index(drop=True)

# protocol_feature = protocol_feature_query(limit=1000000)
# protocol_feature(f"cache_{len(protocol_feature)}.feather")
protocol_feature = pd.read_feather(f"cache_75825.feather")
protocol_feature['_enrollment'] = protocol_feature['_enrollment'].astype(int)

# this is a good chart. Sent to steve
# px.box(
#     protocol_feature,
#     x="_start_yr",
#     y="_arm_count",
#     title=f"n={len(protocol_feature)} Studies",
#     facet_col="_random",
# )


In [25]:
protocol_feature['_location_count_z'] = zscore(protocol_feature['_location_count'].fillna(0)) 
df = protocol_feature[protocol_feature['_phase'].isin(['Phase 2','Phase 3'])].reset_index(drop=True)
df.head()

,_org_name,_sponsor_name,_location,_enrollment,_status,_phase,_arm,_random,_location_count,_arm_count,_start_yr,_end_yr,_last_yr,_enrollment_z,_arm_count_z,_location_count_z
0,M.D. Anderson Cancer Center,M.D. Anderson Cancer Center,"[{'LocationCity': 'Houston', 'LocationContactL...",80,Recruiting,Phase 2,[{'ArmGroupDescription': 'Patients receive NBT...,Non-Randomized,1.0,2.0,2021.0,2025.0,2021,-0.012723,-0.047726,-0.169401
1,Valneva Austria GmbH,Valneva Austria GmbH,"[{'LocationCity': 'Phoenix', 'LocationContactL...",375,Enrolling by invitation,Phase 3,"[{'ArmGroupDescription': None, 'ArmGroupInterv...",N/A,11.0,1.0,2021.0,2025.0,2021,-0.006247,-0.767892,0.117425
2,Memorial Sloan Kettering Cancer Center,Memorial Sloan Kettering Cancer Center,"[{'LocationCity': 'Commack', 'LocationContactL...",260,Recruiting,Phase 2,[{'ArmGroupDescription': 'The intervention wil...,Randomized,4.0,3.0,2021.0,2025.0,2021,-0.008772,0.672440,-0.083353
3,Shanghai Miracogen Inc.,Shanghai Miracogen Inc.,"[{'LocationCity': 'Bengbu', 'LocationContactLi...",86,Recruiting,Phase 2,[{'ArmGroupDescription': 'MRG002 will be admin...,N/A,7.0,1.0,2021.0,2022.0,2021,-0.012591,-0.767892,0.002695
4,Hanmi Pharmaceutical Company Limited,Hanmi Pharmaceutical Company Limited,"[{'LocationCity': 'Seoul', 'LocationContactLis...",116,Recruiting,Phase 3,"[{'ArmGroupDescription': None, 'ArmGroupInterv...",Randomized,1.0,2.0,2020.0,2021.0,2021,-0.011932,-0.047726,-0.169401


In [30]:
# lets look at locations
df = df[df['_location_count_z'] < 3]
df = df.sort_values("_location_count").reset_index(drop=True)
df

,_org_name,_sponsor_name,_location,_enrollment,_status,_phase,_arm,_random,_location_count,_arm_count,_start_yr,_end_yr,_last_yr,_enrollment_z,_arm_count_z,_location_count_z
0,M.D. Anderson Cancer Center,M.D. Anderson Cancer Center,"[{'LocationCity': 'Houston', 'LocationContactL...",80,Recruiting,Phase 2,[{'ArmGroupDescription': 'Patients receive NBT...,Non-Randomized,1.0,2.0,2021.0,2025.0,2021,-0.012723,-0.047726,-0.169401
1,"University of California, San Francisco","University of California, San Francisco","[{'LocationCity': 'SAn Francisco', 'LocationCo...",80,Completed,Phase 3,[{'ArmGroupDescription': 'A randomized placebo...,Randomized,1.0,2.0,2007.0,2007.0,2008,-0.012723,-0.047726,-0.169401
2,Virchow Group,Virchow Group,"[{'LocationCity': 'Hyd', 'LocationContactList'...",100,Completed,Phase 3,[{'ArmGroupDescription': 'TULY (rasburicase) 0...,N/A,1.0,1.0,2010.0,2011.0,2014,-0.012284,-0.767892,-0.169401
3,Takeda,Shire,"[{'LocationCity': 'Philadelphia', 'LocationCon...",280,Completed,Phase 3,None,Randomized,1.0,NaN,2003.0,2005.0,2021,-0.008333,-1.488058,-0.169401
4,Rottapharm Spain,Rottapharm Spain,"[{'LocationCity': 'El Hospitalet', 'LocationCo...",39,Terminated,Phase 2,[{'ArmGroupDescription': 'Megestrol acetate 16...,Randomized,1.0,2.0,2007.0,2009.0,2011,-0.013623,-0.047726,-0.169401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20377,"Janssen-Ortho Inc., Canada","Janssen-Ortho Inc., Canada",None,48,Completed,Phase 3,None,Randomized,NaN,NaN,2004.0,NaN,2011,-0.013425,-1.488058,-0.198084
20378,Merck Sharp & Dohme Corp.,Merck Sharp & Dohme Corp.,None,708,Completed,Phase 2,[{'ArmGroupDescription': 'Participants were va...,Randomized,NaN,4.0,2001.0,2003.0,2015,0.001062,1.392606,-0.198084
20379,Merck Sharp & Dohme Corp.,Merck Sharp & Dohme Corp.,None,111,Completed,Phase 2,[{'ArmGroupDescription': 'Vicriviroc 30 mg QD'...,Randomized,NaN,2.0,2007.0,2010.0,2015,-0.012042,-0.047726,-0.198084
20380,Merck Sharp & Dohme Corp.,Merck Sharp & Dohme Corp.,None,52,Completed,Phase 3,[{'ArmGroupDescription': 'After receiving sevo...,Randomized,NaN,2.0,2006.0,2007.0,2019,-0.013337,-0.047726,-0.198084


In [36]:
loc = df['_location'][0]
pd.json_normalize(loc)

,LocationCity,LocationCountry,LocationFacility,LocationState,LocationStatus,LocationZip,LocationContactList.LocationContact
0,Houston,United States,M D Anderson Cancer Center,Texas,Recruiting,77030,[{'LocationContactEMail': 'JPhan@mdanderson.or...
